Getting weather and AQI data from OpenWeather in json format, merging and converting it into csv

In [19]:
API_KEY = '6dc507f5ca3f2dac7486e5037f6c9f89'

In [20]:
import pandas as pd
import os
import requests as req
import json
import csv
from datetime import datetime, timedelta

In [21]:
aqi = pd.read_csv('AirNowAPI_20223514103516.csv', header=None)

In [22]:
lat = aqi[0][0]
lon = aqi[1][0]

In [23]:
# API Parameters
options = {}
options["aqi_url"] = "http://api.openweathermap.org/data/2.5/air_pollution"
options["url"] = "https://api.openweathermap.org/data/2.5/weather"
options["lat"] = str(lat)
options["lon"] = str(lon)
options["units"] = "metric"
options["api_key"] = API_KEY

In [24]:
def weather_url():
    REQUEST_URL = options["url"] + "?lat=" \
    + options["lat"] \
    + "&lon=" + options["lon"] \
    + "&appid=" + options["api_key"] \
    + "&units=" + options["units"]
    return REQUEST_URL

In [25]:
def aqi_url():
    REQUEST_URL = options["aqi_url"] + "?lat=" \
    + options["lat"] \
    + "&lon=" + options["lon"] \
    + "&appid=" + options["api_key"]
    return REQUEST_URL

In [26]:
print(aqi_url())

http://api.openweathermap.org/data/2.5/air_pollution?lat=21.310301&lon=-157.858093&appid=6dc507f5ca3f2dac7486e5037f6c9f89


In [27]:
print(weather_url())

https://api.openweathermap.org/data/2.5/weather?lat=21.310301&lon=-157.858093&appid=6dc507f5ca3f2dac7486e5037f6c9f89&units=metric


In [28]:
def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour) + timedelta(hours=t.minute//30))

In [29]:
def call_weather_api():
    response = req.get(weather_url())
    data = response.json()
    dw = pd.json_normalize(data['weather'])
    dt = pd.json_normalize(data)
    dt = dt.drop('weather', axis=1)
    df_update = pd.concat([dw,dt],axis=1)

    df_update['dt'] = hour_rounder(datetime.utcfromtimestamp(df_update['dt'])).strftime('%Y-%m-%d %H:%M:%S')
    df_update['sys.sunrise'] = datetime.utcfromtimestamp(df_update['sys.sunrise']).strftime('%Y-%m-%d %H:%M:%S')
    df_update['sys.sunset'] = datetime.utcfromtimestamp(df_update['sys.sunset']).strftime('%Y-%m-%d %H:%M:%S')
    return df_update

In [39]:
def call_aqi_api():
    response = req.get(aqi_url())
    data = response.json()
    dw = pd.json_normalize(data['list'])
    dt = pd.json_normalize(data)
    dt = dt.drop('list', axis=1)
    df_update = pd.concat([dw,dt],axis=1)
    df_update['dt'] = hour_rounder(datetime.utcfromtimestamp(df_update['dt'])).strftime('%Y-%m-%d %H:%M:%S')
    return df_update


In [41]:
df_weather = call_weather_api()
df_aqi = call_aqi_api()

C:\Users\fashaikh\AppData\Local\Temp\ipykernel_13284\3978343323.py:9: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  df_update['dt'] = hour_rounder(datetime.utcfromtimestamp(df_update['dt'])).strftime('%Y-%m-%d %H:%M:%S')
C:\Users\fashaikh\AppData\Local\Temp\ipykernel_13284\3978343323.py:10: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  df_update['sys.sunrise'] = datetime.utcfromtimestamp(df_update['sys.sunrise']).strftime('%Y-%m-%d %H:%M:%S')
C:\Users\fashaikh\AppData\Local\Temp\ipykernel_13284\3978343323.py:11: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  df_update['sys.sunset'] = datetime.utcfromtime

In [48]:
df_merged = pd.merge(df_weather,df_aqi, on=['dt'])
df_merged.to_csv("merged.csv", index=False)

In [10]:
def weather_json_csv():
    response = req.get(weather_url())
    data = response.json()
    dw = pd.json_normalize(data['weather'])
    dt = pd.json_normalize(data)
    dt = dt.drop('weather', axis=1)
    df_update = pd.concat([dw,dt],axis=1)
    df_update.to_csv("weather.csv",index=False)

In [11]:
def aqi_json_csv():
    response = req.get(aqi_url())
    data = response.json()
    dw = pd.json_normalize(data['list'])
    dt = pd.json_normalize(data)
    dt = dt.drop('list', axis=1)
    df_update = pd.concat([dw,dt],axis=1)
    df_update.to_csv("aqi.csv",index=False)